In [ ]:
# 1. 아래페이지에서 순위가 1위부터 5위까지의 영화제목과 링크를 수집
# https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date=20210525

import requests
import bs4

url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date=20210520"
response = requests.get(url)

soup = bs4.BeautifulSoup(response.text, 'html.parser') 

arr = []
for tmp in soup.select("div.tit3", limit=5 ):
    print(tmp.select_one('a').attrs['title'], 'https://movie.naver.com' + tmp.select_one('a').attrs['href'])    
    
    arr.append('https://movie.naver.com/' + tmp.select_one('a').attrs['href'])

arr  # 수집된 영화상세페이지 링크 리스트 

In [ ]:
# 2. 수집된 링크를 이용하여 상세페이지 수집(영화제목, 관람등급, 관객평점, 장르, 상영시간, 개봉일, 감독, 출연배우)
# 이 부분은 위에서 수집한 링크 주소 예시1) https://movie.naver.com/movie/bi/mi/basic.nhn?code=189150

import requests
import bs4

arr3 = []
for tmp in arr:
    url = tmp
    response = requests.get(url)

    soup = bs4.BeautifulSoup(response.text, 'html.parser') 
    
    a = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a:nth-child(1)')
    print("영화제목 : ", a.text)

    b = soup.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a')
    print("장르 : ", b.text)

    c = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p > a")
    print("관람등급 : ", c.text)
    
    d = soup.select_one("#actualPointPersentBasic > div > span > span")
    print("관람객평점 : ", d.text)

    e = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(3)")
    print("상영시간 : ", e.text)
    
    f = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4) > a:nth-child(1)")
    g = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4) > a:nth-child(2)")
    print("개봉일 : " , f.text.strip() +  g.text)

    h = soup.select_one("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4)")
    print("감독 : ", h.text)

    j = []
    for tmp in soup.select("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p"):
        j.append(tmp.text)
        
    print("배우 : ", ','.join(j)); 
    
    print("=" * 30)
        
    arr3.append({"a":a.text, "b":b.text,"c":c.text,"d":d.text,"e":e.text,"f":f.text.strip() +  g.text, "h":h.text, "j": ','.join(j)})


#----------------------
arr3

In [ ]:
# arr4 = arr3  # 내용은 같으면서 객체도 같은

import copy
arr4 = copy.deepcopy(arr3)

# arr3과 arr4는 내용은 같으면서 객체는 다른 객체

# arr3 => mongo
# arr4 => hadoop

In [ ]:
from selenium import webdriver  # pip install selenium
import bs4 # pip install bs4
import time
import urllib.request as REQ  #서버 요청

# 옵션설정
options = webdriver.ChromeOptions()
options.add_argument("disable-gpu")   
options.add_argument("lang=ko_KR")  
# options.add_argument("headless")  # 크롬창이 표시되지 않음
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')  

# 드라이브 로딩
driver = webdriver.Chrome(executable_path="driver/chromedriver.exe", options=options)

for tmp in arr:
    url = tmp
    
    # 페이지 접속
    driver.get(url)
    
    # 로딩되는 시간이 있으므로 1초 기다림
    time.sleep(1)

    img = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/a/img')
    print(img.get_attribute("src"))
    
    title = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]').text
    
    # 제목에 : 있으면 파일저장 : 공백으로 처리
    title = title.replace(":", "")
    
    #서버요청(이미지의 url주소, 저장할 파일명)
    REQ.urlretrieve( img.get_attribute("src"), f'download/{title}.jpg' )
    print("-" * 30)

driver.close()

In [ ]:
import pymongo

try:
    username = "id100"
    password = "pw100"
    host = "1.234.5.158"
    port = 37017
    dbname = "id100"
    
    conn = pymongo.MongoClient(f'mongodb://{username}:{password}@{host}:{port}/{dbname}')
    db = conn.get_database(dbname)
    collection = db.get_collection("exam_movie")

    # 여러개 추가하기
    collection.insert_many(arr3)
except Exception as e:
    print(e)
finally:
    conn.close()

In [ ]:
import hdfs
import json

client_hdfs = hdfs.InsecureClient('http://1.234.5.158:9870', user='hdfs')

client_hdfs.delete('/id100/exam_movie.json')
with client_hdfs.write('/id100/exam_movie.json', encoding = "utf-8" ) as writer:
    json.dump(arr4, writer)

In [ ]:
import pandas as pd

with client_hdfs.read('/id100/exam_movie.json', encoding = 'utf-8') as reader:
    df2 = pd.read_json(reader)
    
df2 